<a href="https://colab.research.google.com/github/lifepopkay/Tech-Monies/blob/Modelling/Third_Model_Filling_upper_salary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the required libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
url = 'https://raw.githubusercontent.com/lifepopkay/Tech-Monies/Modelling/Mergered%20Cleaned%20Data%203.csv'
df = pd.read_csv(url)

In [3]:
df.head(2)

id       company location     postDate title scraped for  \
0  job_3e1d31dda6c5cb4b      Offerzen    Lagos  Just posted    Data Scientist   
1  job_c3e2ed6ba483528b  GVA Partners    Lagos        Today    Data Scientist   

   Country  State  age      Position  upper_salary  lower_salary  \
0  Nigeria  Lagos  NaN  Senior Level           NaN           NaN   
1  Nigeria  Lagos  NaN           NaN           NaN           NaN   

  payment_frequency  year_of_exp contract_type    eligibility  \
0               NaN          2.0     Full-time            NaN   
1               NaN          4.0           NaN  undergraduate   

              title_cleaned                                        skills_list  
0  Senior Software Engineer                      ['asp', 'C', 'R', 'D', 'AWS']  
1            Data Scientist  ['Regression', 'C', 'KNN', 'Python', 'D', 'asp...

In [4]:
# Standardizing the title 
ML=['Machine Learning', 'Machine Learning ']
for title in ML:
    df['title scraped for'].replace(title,'Machine Learning Engineer', inplace = True)

In [5]:
# Dropping records for canada
df = df.query("Country != 'Canada'")
# Dropping records for data engineering
df = df[df["title scraped for"] != "Data Engineer"]

# Columns to drop
columns_to_drop = ["company", "location", "State", "age", "id","postDate"]

In [6]:
# Drop unnecessary columns
df.drop(columns_to_drop, axis="columns", inplace = True)

In [7]:
# Defining a function that replaces all values o years of eperience greater than 30 with np.nan
def yoe_greater_than_30(yoe):
    if yoe > 30:
        return np.nan
    else:
        return yoe
df.year_of_exp = df.year_of_exp.apply(yoe_greater_than_30)

In [8]:
# Replace "hs" with "high school diploma" in eligibility
df.eligibility.replace("hs",'high school diploma', inplace = True)

# Fill missing values for eligibility with "high school diploma"
df.eligibility.fillna('high school diploma', inplace = True)

In [9]:
# Fill missing values for contract_type with the mode of contract_type
#df.contract_type.fillna(df.contract_type.mode()[0], inplace = True)

In [10]:
# Creating an exchange rate for each country currency with respect to UD dollars
Exchange_rate = {"Nigeria": 427.88,
"India": 79.97,
"USA": 1,
"UK": 1.17}
df["Exchange_rate"] = df.Country.map(Exchange_rate)

# Converting the salary currency for each Job to US dollars
df.upper_salary = df.upper_salary / df.Exchange_rate
df.lower_salary = df.lower_salary / df.Exchange_rate

df.drop('Exchange_rate', axis= "columns", inplace = True)

On exploring the salary for missing Payment frequency, it was discovered that the salary far exceeds the salary for hourly payment frequency and is far below the salary for minthly payment frequency. 

So the the missing values for payment frequency will be filled with "Weekly"

In [11]:
# Filling the missing values for payment frequency with "Weekly"
df.payment_frequency.fillna("Weekly", inplace = True)

In [12]:
# Defining a function that cleans the skills column
def convert_skill_list_to_list(skills):
    skills = skills.replace("[", "")
    skills = skills.replace("]","")
    skills = skills.replace("'", "")
    Skills_list = skills.split(",")
    Skills_lists = []
    for skill in Skills_list:
        skill = skill.strip()
        Skills_lists.append(skill)
    return Skills_lists

In [13]:
# Converting the skills list to an actaul list
df.skills_list = df.skills_list.apply(convert_skill_list_to_list)

In [14]:
# Defining a function that counts the number of job posting that require a particular skill
def Skills_table(df):
    list_skill = {}
    for Skill_list in df.skills_list:
        for Skill in Skill_list:
            if Skill in list_skill:
                list_skill[Skill] += 1
            else:
                list_skill[Skill] = 1
    skill_df = pd.DataFrame(list_skill, index = ["Number of jobs"]).T.sort_values("Number of jobs", ascending = False)
    return skill_df

In [15]:
# Selecting the Job postings for data analyst roles
DA_df = df[df["title scraped for"] == "Data Analyst"]

# Selecting the Job postings for data scientist roles
DS_df = df[df["title scraped for"] == "Data Scientist"]

# Selecting the Job postings for business analyst roles
BA_df = df[df["title scraped for"] == "Business Analyst"]

# Selecting the Job postings for machine learning engineering roles
ML_df = df[df["title scraped for"] == "Machine Learning Engineer"]

# Selecting the Job postings for web developer roles
WD_df = df[df["title scraped for"] == "Web Developer"]

# Selecting the Skills required for Data analyst roles
DASkills_df = Skills_table(DA_df)

# Saving the top 10 skills for Data Analyst to a list
DAT10_skills = list(DASkills_df.head(10).index)

# Selecting the Skills required for Data scientist roles
DSSkills_df = Skills_table(DS_df)

# Saving the top 10 skills for Data Scientist to a list
DST10_skills = list(DSSkills_df.head(10).index)

# Selecting the Skills required for Business analyst roles
BASkills_df = Skills_table(BA_df)

# Saving the top 10 skills for Business Analyst to a list
BAT10_skills = list(BASkills_df.head(10).index)

# Selecting the Skills required for Machine learning roles
MLSkills_df = Skills_table(ML_df)

# Saving the top 10 skills for Machine learning to a list
MLT10_skills = list(MLSkills_df.head(10).index)

# Selecting the Skills required for Web developer roles
WDSkills_df = Skills_table(WD_df)

# Saving the top 10 skills for Web developer to a list
WDT10_skills = list(WDSkills_df.head(10).index)

# Merging the top 10 skills for each role into a set
Most_popular_skills = set(DAT10_skills + DST10_skills + BAT10_skills + MLT10_skills + WDT10_skills)

len(list(Most_popular_skills))

22

In [16]:
# Dropping missing values for title scraped for
df.dropna(subset = ["title scraped for"], inplace = True)

In [17]:
# Dropping records with missing values for lower salary range
df.dropna(subset=["lower_salary"], inplace = True)

In [18]:
# Defining a function that counts the number of recors we have for each level for each role for each country
def number_of_observations(data):
    return data.groupby(["Country","title scraped for","Position"]).agg({"Position":"count"})

number_of_observations(df)

Position
Country title scraped for         Position              
India   Business Analyst          Mid-Level            2
                                  Senior Level         5
        Data Analyst              Senior Level         7
        Data Scientist            Mid-Level            4
                                  Senior Level        24
        Machine Learning Engineer Mid-Level            4
                                  Senior Level        17
        Web Developer             Mid-Level            9
                                  Senior Level        54
Nigeria Business Analyst          Senior Level         5
        Data Analyst              Senior Level         2
        Data Scientist            Senior Level         1
        Machine Learning Engineer Mid-Level            1
                                  Senior Level         5
        Web Developer             Mid-Level            2
                                  Senior Level         2
UK      Business Analyst          Mid-Level            1
                                  Senior Level        61
        Data Analyst              Mid-Level            1
                                  Senior Level       458
        Data Scientist            Entry Level          1
                                  Mid-Level           13
                                  Senior Level       171
        Machine Learning Engineer Mid-Level            3
                                  Senior Level        51
        Web Developer             Mid-Level           40
                                  Senior Level       119
USA     Business Analyst          Mid-Level            6
                                  Senior Level       128
        Data Analyst              Entry Level          3
                                  Mid-Level            9
                                  Senior Level        44
        Data Scientist            Entry Level          1
                                  Mid-Level            7
                                  Senior Level       266
        Machine Learning Engineer Entry Level          3
                                  Mid-Level           10
                                  Senior Level        98
        Web Developer             Entry Level          2
                                  Mid-Level           15
                                  Senior Level        54

In [19]:
# Determining the average years of experience for each role and position
df.groupby(["title scraped for","Position"])[["year_of_exp"]].agg(np.mean)

year_of_exp
title scraped for         Position                 
Business Analyst          Mid-Level        3.750000
                          Senior Level     4.162162
Data Analyst              Entry Level      1.000000
                          Mid-Level        3.428571
                          Senior Level     3.947368
Data Scientist            Entry Level      1.000000
                          Mid-Level        2.785714
                          Senior Level     3.666667
Machine Learning Engineer Entry Level      3.500000
                          Mid-Level        3.750000
                          Senior Level     6.000000
Web Developer             Entry Level      5.000000
                          Mid-Level        2.833333
                          Senior Level     3.590278

In [20]:
# Defining a function that determines the Position based on years of expereience
def Position_based_on_yoe(data):
    data.reset_index(drop=True, inplace = True)
    for i in range(data.shape[0]):
        if  data.loc[i,"Position"] is np.nan:
            if data.loc[i,"title scraped for"] == 'Business Analyst':
                if  data.loc[i,"year_of_exp"] >= 3 and data.loc[i,"year_of_exp"] <= 5:
                        data.loc[i,"Position"] = "Mid-Level"
                elif data.loc[i,"year_of_exp"] > 5:
                        data.loc[i,"Position"] = "Senior Level"
                elif data.loc[i,"year_of_exp"] < 3:
                    data.loc[i,"Position"] = "Entry Level"

            elif data.loc[i,"title scraped for"] == 'Data Analyst':
                if  data.loc[i,"year_of_exp"] > 2 and data.loc[i,"year_of_exp"] <= 4:
                        data.loc[i,"Position"] = "Mid-Level"
                elif data.loc[i,"year_of_exp"] > 4:
                        data.loc[i,"Position"] = "Senior Level"
                elif data.loc[i,"year_of_exp"] <= 2:
                    data.loc[i,"Position"] = "Entry Level"

            elif data.loc[i,"title scraped for"]== 'Data Scientist':
                if  data.loc[i,"year_of_exp"] > 2 and data.loc[i,"year_of_exp"] <= 4:
                        data.loc[i,"Position"] = "Mid-Level"
                elif data.loc[i,"year_of_exp"] > 4:
                        data.loc[i,"Position"] = "Senior Level"
                elif data.loc[i,"year_of_exp"] <= 2:
                    data.loc[i,"Position"] = "Entry Level"
            elif data.loc[i,"title scraped for"] == 'Machine Learning Engineer':
                if  data.loc[i,"year_of_exp"] > 3 and data.loc[i,"year_of_exp"] <= 5:
                        data.loc[i,"Position"] = "Mid-Level"
                elif data.loc[i,"year_of_exp"] > 5:
                        data.loc[i,"Position"] = "Senior Level"
                elif data.loc[i,"year_of_exp"] <= 3:
                    data.loc[i,"Position"] = "Entry Level"

            elif data.loc[i,"title scraped for"] == 'Web Developer':
                if  data.loc[i,"year_of_exp"] >= 2 and data.loc[i,"year_of_exp"] <= 4:
                        data.loc[i,"Position"] = "Mid-Level"
                elif data.loc[i,"year_of_exp"] > 4:
                        data.loc[i,"Position"] = "Senior Level"
                elif data.loc[i,"year_of_exp"] < 2:
                    data.loc[i,"Position"] = "Entry Level"
        
        else:
            data.loc[i,"Position"] = data.loc[i,"Position"]
    return data

In [21]:
# Making a copy of the data frame
data = df.copy()

# Determining the position of the job role using the Position_based_on_yoe function
df = Position_based_on_yoe(data)

In [22]:
# Defining a function that counts the number of recors we have for each level for each role for each country
def number_of_observations(data):
    return data.groupby(["Country","title scraped for","Position"]).agg({"Position":"count"})

number_of_observations(df)

Position
Country title scraped for         Position              
India   Business Analyst          Entry Level         38
                                  Mid-Level           20
                                  Senior Level        10
        Data Analyst              Entry Level         34
                                  Mid-Level           10
                                  Senior Level        15
        Data Scientist            Entry Level         53
                                  Mid-Level           22
                                  Senior Level        40
        Machine Learning Engineer Entry Level         69
                                  Mid-Level           26
                                  Senior Level        29
        Web Developer             Entry Level        194
                                  Mid-Level          126
                                  Senior Level        67
Nigeria Business Analyst          Entry Level          2
                                  Mid-Level           21
                                  Senior Level         5
        Data Analyst              Entry Level          1
                                  Mid-Level            8
                                  Senior Level         6
        Data Scientist            Mid-Level            1
                                  Senior Level         2
        Machine Learning Engineer Mid-Level            2
                                  Senior Level         5
        Web Developer             Entry Level          5
                                  Mid-Level           14
                                  Senior Level         6
UK      Business Analyst          Mid-Level          386
                                  Senior Level        61
        Data Analyst              Mid-Level            1
                                  Senior Level       458
        Data Scientist            Entry Level         33
                                  Mid-Level           33
                                  Senior Level       466
        Machine Learning Engineer Mid-Level            3
                                  Senior Level       316
        Web Developer             Entry Level         19
                                  Mid-Level           99
                                  Senior Level       143
USA     Business Analyst          Entry Level         30
                                  Mid-Level          579
                                  Senior Level       145
        Data Analyst              Entry Level         47
                                  Mid-Level           41
                                  Senior Level        64
        Data Scientist            Entry Level         31
                                  Mid-Level          401
                                  Senior Level       294
        Machine Learning Engineer Entry Level        102
                                  Mid-Level           52
                                  Senior Level       137
        Web Developer             Entry Level         22
                                  Mid-Level           65
                                  Senior Level        92

In [23]:
# Number of missing values for Position. These indicates Positions for job roles with missing values for years of experience
df.Position.isna().sum()

808

In [24]:
Frequency_to_yearly = {"Hourly": 40 * 52,
"Weekly": 52,
"Monthly": 12,
"Yearly": 1}
df["Frequency_to_yearly"] = df.payment_frequency.map(Frequency_to_yearly)

# Standardizing the payment frequency to Yearly
df.upper_salary = df.upper_salary * df["Frequency_to_yearly"]
df.lower_salary = df.lower_salary * df["Frequency_to_yearly"]
df.drop('Frequency_to_yearly', axis= "columns", inplace = True)

In [25]:
# Dropping unwanted columns
df.drop(['title_cleaned','payment_frequency'], axis="columns", inplace = True)

In [26]:
#df.dropna(subset=["upper_salary"], inplace = True)

In [27]:
# Determining the average upper and lower salary for each level for each title for each country
df.groupby(["Country", "title scraped for", "Position"])[["lower_salary","upper_salary"]].agg(np.mean)

lower_salary   upper_salary
Country title scraped for         Position                                  
India   Business Analyst          Entry Level     3977.582383    7167.303755
                                  Mid-Level       8195.926597   16694.828894
                                  Senior Level   14283.956484   20863.223709
        Data Analyst              Entry Level     3941.339767    6863.545258
                                  Mid-Level       5337.035138   11998.191822
                                  Senior Level   15578.241841   26181.279518
        Data Scientist            Entry Level     7370.231054   14423.958985
                                  Mid-Level       7571.778053   15963.670752
                                  Senior Level   16775.040640   28908.425129
        Machine Learning Engineer Entry Level     9310.508288   12973.596458
                                  Mid-Level      12556.005617   20302.353263
                                  Senior Level   16162.009029   28012.908687
        Web Developer             Entry Level     3331.118306    6622.027615
                                  Mid-Level       6772.828600   10143.233637
                                  Senior Level    7648.726481   13120.181607
Nigeria Business Analyst          Entry Level     3505.655791            NaN
                                  Mid-Level       7472.054915   11942.600729
                                  Senior Level   15985.790409   12620.360849
        Data Analyst              Entry Level     2804.524633            NaN
                                  Mid-Level       6723.847808    7992.895204
                                  Senior Level   10049.546602   20099.093204
        Data Scientist            Mid-Level      19631.672432            NaN
                                  Senior Level    5609.049266            NaN
        Machine Learning Engineer Mid-Level       4206.786950            NaN
                                  Senior Level    4487.239413    7852.668973
        Web Developer             Entry Level     2832.569879    5609.049266
                                  Mid-Level       6610.665207    7946.153127
                                  Senior Level   12262.004923   26713.097130
UK      Business Analyst          Mid-Level      34472.101324   39475.532544
                                  Senior Level   41273.448228   51241.564474
        Data Analyst              Mid-Level      55555.555556   85470.085470
                                  Senior Level   34595.680961   43543.118363
        Data Scientist            Entry Level    48974.747475   62348.553583
                                  Mid-Level      52204.014504   55428.576737
                                  Senior Level   49833.157624   63350.063472
        Machine Learning Engineer Mid-Level      45584.045584   47008.547009
                                  Senior Level   42834.850698   53293.836013
        Web Developer             Entry Level   111129.554656   36868.888889
                                  Mid-Level      54222.595183   61754.035574
                                  Senior Level   43813.364413   55341.097532
USA     Business Analyst          Entry Level    71646.666667   91435.333333
                                  Mid-Level      75873.553022   96417.677273
                                  Senior Level   89781.788966  114852.077037
        Data Analyst              Entry Level    68718.361702   85928.608696
                                  Mid-Level      68509.434146   87205.070000
                                  Senior Level   84156.425000  108995.219672
        Data Scientist            Entry Level   101763.258065  133110.200000
                                  Mid-Level     103565.383042  130790.239554
                                  Senior Level  108379.372109  139339.406338
        Machine Learning Engineer Entry Level   101019.713725  128220.010753
                                  Mid-Level  

In [28]:
# Populating the each skills with 1 if a particular role required the skill and 0 if otherwise
i = 0
for skills_list in df.skills_list:
        for skill in skills_list:
            if skill in Most_popular_skills:
            #skill = skill.strip()
                df.loc[i, skill] = 1
        i +=1

In [29]:
# Dropping the skills list column
df.drop(["skills_list"], axis="columns", inplace = True)

In [30]:
# Adding the most popular skills to the dataframe
for Skill in list(Most_popular_skills):
    df[Skill] = 0

In [31]:
df.head()

title scraped for  Country      Position  upper_salary  lower_salary  \
0    Data Scientist  Nigeria     Mid-Level           NaN  19631.672432   
1    Data Scientist  Nigeria  Senior Level           NaN   5609.049266   
2    Data Scientist  Nigeria  Senior Level           NaN   5609.049266   
3     Web Developer  Nigeria  Senior Level  37393.661774  21033.934748   
4     Web Developer  Nigeria  Senior Level  39730.765635  23371.038609   

   year_of_exp contract_type    eligibility  R  Spark  ...  HTML  Excel  Go  \
0          3.0     Full-time  undergraduate  0      0  ...     0      0   0   
1          5.0     Full-time  undergraduate  0      0  ...     0      0   0   
2          6.0     Full-time  undergraduate  0      0  ...     0      0   0   
3          5.0     Full-time  undergraduate  0      0  ...     0      0   0   
4          5.0     Full-time  undergraduate  0      0  ...     0      0   0   

   PostgreSQL  Vi  client  ETL  Machine Learning     SDLC  
0           0   0       0    0                 0  0     0  
1           0   0       0    0                 0  0     0  
2           0   0       0    0                 0  0     0  
3           0   0       0    0                 0  0     0  
4           0   0       0    0                 0  0     0  

[5 rows x 30 columns]

### Categorical Column Coding map

In [32]:
# Making a copy of the data
df7 = df.copy()
# Encoding the categorical variables
df6 = df.copy()
categorical_columns = ["title scraped for",'Country','Position','contract_type','eligibility']
df[categorical_columns] = df[categorical_columns].astype('category')
df6 = df.copy()
for x in categorical_columns:
        df[x] = df[x].cat.codes
   

In [33]:
# Saving the codes for Variable
Title_scrapped_for  = dict(enumerate(df6['title scraped for'].cat.categories))
COUNTRY  = dict(enumerate(df6['Country'].cat.categories))
POSITION = dict(enumerate(df6['Position'].cat.categories))
CONTRACT_TYPE = dict(enumerate(df6['contract_type'].cat.categories))
ELIGIBILTY = dict(enumerate(df6['eligibility'].cat.categories))

In [34]:
# Restoring the actual values for position using the duplicate data
df.Position = df7.Position

# Confirming the Position column has been restored
df.Position.isna().sum()

808

In [35]:
# Code or mapping for Title Scraped for
Title_scrapped_for

{0: 'Business Analyst',
 1: 'Data Analyst',
 2: 'Data Scientist',
 3: 'Machine Learning Engineer',
 4: 'Web Developer'}

In [36]:
# Code or mapping for Country
COUNTRY

{0: 'India', 1: 'Nigeria', 2: 'UK', 3: 'USA'}

In [37]:
# Code or mapping for Position
POSITION

{0: 'Entry Level', 1: 'Mid-Level', 2: 'Senior Level'}

In [38]:
# Code or mapping for Contract type
CONTRACT_TYPE

{0: 'Contract', 1: 'Full-time'}

In [39]:
# Code or mapping for Eligibility
ELIGIBILTY

{0: 'doctorate',
 1: 'high school diploma',
 2: 'postgraduate',
 3: 'undergraduate'}

In [40]:
# Making a duplicate of the data
df1 = df.copy()

In [41]:
# Subsetting the jobs for India
IND_JOBS = df1.query("Country == 0")
# Subsetting the jobs for Nigeria
NIG_JOBS = df1.query("Country == 1")
# Subsetting the jobs for UK
UK_JOBS  = df1.query("Country == 2")
# Subsetting the jobs for US
USA_JOBS = df1.query("Country == 3")
# Subsetting the jobs for India, UK and US
UK_IND_US_JOBS = df1.query("Country !=1")

In [42]:
# Checking the missing values in the UK_IND_US_JOBS data
UK_IND_US_JOBS.isna().sum()

title scraped for       0
Country                 0
Position              771
upper_salary          937
lower_salary            0
year_of_exp          1502
contract_type           0
eligibility             0
R                       0
Spark                   0
C++                     0
Java                    0
SQL                     0
C                       0
Python                  0
D                       0
AWS                     0
Git                     0
ML                      0
CSS                     0
HTML                    0
Excel                   0
Go                      0
PostgreSQL              0
Vi                      0
client                  0
ETL                     0
Machine Learning        0
                        0
SDLC                    0
dtype: int64

In [43]:
# Reseting the index for UK_IND_US_JOBS
UK_IND_US_JOBS.reset_index(drop=True, inplace = True)
# Making a duplicate for UK_IND_US_JOBS
UK_IND_US_JOBS_2 = UK_IND_US_JOBS.copy()
# Preserving Position, Upper and lower salary columns by dropping them
UK_IND_US_JOBS.drop(["upper_salary", "lower_salary", "Position"], axis="columns", inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [44]:
# Fill the missing values for Years of experience using an iterative imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute       import IterativeImputer
#from catboost             import CatBoostRegressor

imputer = IterativeImputer(
                            verbose   = 2,
                            max_iter  = 20,
                            imputation_order = 'descending' )


UK_IND_US_JOBS = pd.DataFrame(imputer.fit_transform(UK_IND_US_JOBS), columns = UK_IND_US_JOBS.columns) # preserve the column nam

[IterativeImputer] Completing matrix with shape (5644, 27)
[IterativeImputer] Ending imputation round 1/20, elapsed time 0.29
[IterativeImputer] Change: 3.178072253485904, scaled tolerance: 0.03 
[IterativeImputer] Ending imputation round 2/20, elapsed time 0.56
[IterativeImputer] Change: 0.0, scaled tolerance: 0.03 
[IterativeImputer] Early stopping criterion reached.


In [45]:
# Restoring the Position, uppper and lower salary
UK_IND_US_JOBS_2[list(UK_IND_US_JOBS.columns)] = UK_IND_US_JOBS

In [46]:
# Defining a function that determines the Position based on years of expereience
def Position_based_on_yoe(data):
    data.reset_index(drop=True, inplace = True)
    for i in range(data.shape[0]):
        if  data.loc[i,"Position"] is np.nan:
            if data.loc[i,"title scraped for"] == 0:
                if  data.loc[i,"year_of_exp"] >= 3 and data.loc[i,"year_of_exp"] <= 5:
                        data.loc[i,"Position"] = "Mid-Level"
                elif data.loc[i,"year_of_exp"] > 5:
                        data.loc[i,"Position"] = "Senior Level"
                elif data.loc[i,"year_of_exp"] < 3:
                    data.loc[i,"Position"] = "Entry Level"

            elif data.loc[i,"title scraped for"] == 1:
                if  data.loc[i,"year_of_exp"] > 2 and data.loc[i,"year_of_exp"] <= 5:
                        data.loc[i,"Position"] = "Mid-Level"
                elif data.loc[i,"year_of_exp"] > 5:
                        data.loc[i,"Position"] = "Senior Level"
                elif data.loc[i,"year_of_exp"] <= 2:
                    data.loc[i,"Position"] = "Entry Level"

            elif data.loc[i,"title scraped for"]== 2:
                if  data.loc[i,"year_of_exp"] > 2 and data.loc[i,"year_of_exp"] <= 4:
                        data.loc[i,"Position"] = "Mid-Level"
                elif data.loc[i,"year_of_exp"] > 4:
                        data.loc[i,"Position"] = "Senior Level"
                elif data.loc[i,"year_of_exp"] <= 2:
                    data.loc[i,"Position"] = "Entry Level"
            elif data.loc[i,"title scraped for"] == 3:
                if  data.loc[i,"year_of_exp"] >= 3 and data.loc[i,"year_of_exp"] <= 5:
                        data.loc[i,"Position"] = "Mid-Level"
                elif data.loc[i,"year_of_exp"] > 5:
                        data.loc[i,"Position"] = "Senior Level"
                elif data.loc[i,"year_of_exp"] < 3:
                    data.loc[i,"Position"] = "Entry Level"

            elif data.loc[i,"title scraped for"] == 4:
                if  data.loc[i,"year_of_exp"] >= 2 and data.loc[i,"year_of_exp"] <= 4:
                        data.loc[i,"Position"] = "Mid-Level"
                elif data.loc[i,"year_of_exp"] > 4:
                        data.loc[i,"Position"] = "Senior Level"
                elif data.loc[i,"year_of_exp"] < 2:
                    data.loc[i,"Position"] = "Entry Level"
        
        else:
            data.loc[i,"Position"] = data.loc[i,"Position"]
    return data

In [47]:
# Making a duplicate of the data
data = UK_IND_US_JOBS_2.copy()
# Filling the missing values for Position using the Position_based_on_yoe
UK_IND_US_JOBS_3 = Position_based_on_yoe(data)

In [48]:
number_of_observations(UK_IND_US_JOBS_3)

Position
Country title scraped for Position              
0.0     0.0               Entry Level         47
                          Mid-Level           25
                          Senior Level        10
        1.0               Entry Level         35
                          Mid-Level           23
                          Senior Level        15
        2.0               Entry Level         57
                          Mid-Level          144
                          Senior Level        47
        3.0               Entry Level         76
                          Mid-Level           36
                          Senior Level        29
        4.0               Entry Level        200
                          Mid-Level          153
                          Senior Level        81
2.0     0.0               Mid-Level          387
                          Senior Level        61
        1.0               Mid-Level            1
                          Senior Level       458
        2.0               Entry Level         33
                          Mid-Level           70
                          Senior Level       563
        3.0               Mid-Level            3
                          Senior Level       316
        4.0               Entry Level         19
                          Mid-Level          115
                          Senior Level       317
3.0     0.0               Entry Level         33
                          Mid-Level          616
                          Senior Level       145
        1.0               Entry Level         47
                          Mid-Level           75
                          Senior Level        76
        2.0               Entry Level         31
                          Mid-Level          405
                          Senior Level       319
        3.0               Entry Level        106
                          Mid-Level           86
                          Senior Level       167
        4.0               Entry Level         22
                          Mid-Level           69
                          Senior Level       126

In [49]:
# Encoding the Position column
UK_IND_US_JOBS_3["Position"] = UK_IND_US_JOBS_3["Position"].astype('category')
UK_IND_US_JOBS_3["Position"] = UK_IND_US_JOBS_3["Position"].cat.codes

In [50]:
# Fill the missing values for upper salary using an iterative imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute       import IterativeImputer
#from catboost             import CatBoostRegressor

imputer = IterativeImputer(
                            verbose   = 2,
                            max_iter  = 20,
                            imputation_order = 'descending' )


df2 = pd.DataFrame(imputer.fit_transform(UK_IND_US_JOBS_3), columns = UK_IND_US_JOBS_3.columns) # preserve the column nam

[IterativeImputer] Completing matrix with shape (5644, 30)
[IterativeImputer] Ending imputation round 1/20, elapsed time 0.36
[IterativeImputer] Change: 882040.1868253917, scaled tolerance: 1600.0 
[IterativeImputer] Ending imputation round 2/20, elapsed time 0.71
[IterativeImputer] Change: 0.0, scaled tolerance: 1600.0 
[IterativeImputer] Early stopping criterion reached.


In [51]:
UK_IND_US_JOBS_3.head()

title scraped for  Country  Position  upper_salary  lower_salary  \
0                3.0      0.0         0           NaN   2250.844067   
1                3.0      0.0         0  10003.751407   5001.875703   
2                3.0      0.0         0  22508.440665   7502.813555   
3                3.0      0.0         1           NaN   2250.844067   
4                3.0      0.0         0   4376.641240   2500.937852   

   year_of_exp  contract_type  eligibility    R  Spark  ...  HTML  Excel   Go  \
0     1.000000           -1.0          3.0  0.0    0.0  ...   0.0    0.0  0.0   
1     1.000000            1.0          3.0  0.0    0.0  ...   0.0    0.0  0.0   
2     2.000000            1.0          3.0  0.0    0.0  ...   0.0    0.0  0.0   
3     4.247574           -1.0          1.0  0.0    0.0  ...   0.0    0.0  0.0   
4     2.022137            1.0          2.0  0.0    0.0  ...   0.0    0.0  0.0   

   PostgreSQL   Vi  client  ETL  Machine Learning       SDLC  
0         0.0  0.0     0.0  0.0               0.0  0.0   0.0  
1         0.0  0.0     0.0  0.0               0.0  0.0   0.0  
2         0.0  0.0     0.0  0.0               0.0  0.0   0.0  
3         0.0  0.0     0.0  0.0               0.0  0.0   0.0  
4         0.0  0.0     0.0  0.0               0.0  0.0   0.0  

[5 rows x 30 columns]

In [52]:
#df2 = UK_IND_US_JOBS_3.dropna(subset=["upper_salary"])

In [53]:
# Determining the average upper and lower salary for each level for each title for each country
df2.groupby(["Country", "title scraped for", "Position"])[["lower_salary","upper_salary"]].agg(np.mean)

lower_salary   upper_salary
Country title scraped for Position                              
0.0     0.0               0.0         4011.428754   10593.409671
                          1.0         7853.227460   17905.430531
                          2.0        14283.956484   20863.223709
        1.0               0.0         3953.776952   10071.793156
                          1.0         7087.455078   17234.886129
                          2.0        15578.241841   27083.984729
        2.0               0.0         7044.321664   15491.957465
                          1.0         9207.503682   18056.035460
                          2.0        15571.796871   27750.039394
        3.0               0.0         8913.660550   16423.473194
                          1.0        10165.860114   21155.755099
                          2.0        16162.009029   28097.881923
        4.0               0.0         3310.716581    9989.204680
                          1.0         6331.305204   13928.510793
                          2.0         6850.531912   15722.721656
2.0     0.0               1.0        34453.699066   44816.493396
                          2.0        41273.448228   52720.694996
        1.0               1.0        55555.555556   85470.085470
                          2.0        34595.680961   45001.153062
        2.0               0.0        48974.747475   64767.243251
                          1.0        46161.269841   61087.143588
                          2.0        49518.040109   63877.942839
        3.0               1.0        45584.045584   56862.526677
                          2.0        42834.850698   55206.567483
        4.0               0.0       111129.554656   39883.738145
                          1.0        50316.352285   62224.340719
                          2.0        39195.818167   52489.865386
3.0     0.0               0.0        71757.575758   91692.727273
                          1.0        75759.341234   96680.040046
                          2.0        89781.788966  115367.523652
        1.0               0.0        68718.361702   85528.790321
                          1.0        69930.490667   88339.767900
                          2.0        81229.736842  104340.654745
        2.0               0.0       101763.258065  134077.961714
                          1.0       103276.628642  132254.191749
                          2.0       107009.932288  137893.122379
        3.0               0.0       100486.894340  128580.254233
                          1.0        94647.883721  123428.622029
                          2.0       114029.667066  145157.221531
        4.0               0.0        78665.800000  102388.173410
                          1.0        83695.017391  108291.350502
                          2.0        98113.690476  123174.566308

### Features

In [54]:
# Saving all the features used in developing the model to a dictionary in the order they were used
features_dict = {}
i = 0
for feature in list(df2.drop(["lower_salary", "upper_salary"], axis="columns").columns):
    features_dict[i] = feature
    i +=1
    
print(features_dict)

{0: 'title scraped for', 1: 'Country', 2: 'Position', 3: 'year_of_exp', 4: 'contract_type', 5: 'eligibility', 6: 'R', 7: 'Spark', 8: 'C++', 9: 'Java', 10: 'SQL', 11: 'C', 12: 'Python', 13: 'D', 14: 'AWS', 15: 'Git', 16: 'ML', 17: 'CSS', 18: 'HTML', 19: 'Excel', 20: 'Go', 21: 'PostgreSQL', 22: 'Vi', 23: 'client', 24: 'ETL', 25: 'Machine Learning', 26: '', 27: 'SDLC'}


In [55]:
# Extracting the features and target variable
y = df2[["lower_salary", "upper_salary"]].values
X = df2.drop(["lower_salary", "upper_salary"], axis="columns")
XX = df2.drop(["lower_salary", "upper_salary"], axis="columns")

In [56]:
# Normalizing the data
X= StandardScaler().fit(X).transform(X)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (4515, 28) (4515, 2)
Test set: (1129, 28) (1129, 2)


### Using Linear Regression

In [57]:
Lr = LinearRegression()
Lr.fit(X_train,y_train)

LinearRegression()

In [58]:
yhat = Lr.predict(X_train)
r2_score(y_train, yhat)

0.3894589077780793

In [59]:
yhat_test = Lr.predict(X_test)
r2_score(y_test, yhat_test)

0.6193733022118144

In [60]:
mean_squared_error(y_train, yhat)

1878897533.7954674

### Saving the model

### Using Rigdge Regression model

In [61]:
# Determing the optimum alpha for the ridge regression model
from sklearn.linear_model import Ridge
rscore = []
for alpha in range(1000):
    ridge_reg = Ridge(alpha=alpha)
    ridge_reg.fit(X_train,y_train)
    yhat = ridge_reg.predict(X_train)
    rscore_ = r2_score(y_train, yhat)
    rscore.append(rscore_)
rscore = np.array(rscore)

In [62]:
# Making a plot of the rscore of each alpha from 0 to 1000
import matplotlib.pyplot as plt
plt.plot(rscore)
plt.xlabel("Alpha")
plt.ylabel("rscore")
plt.show()
plt.clf()

<Figure size 432x288 with 0 Axes>

In [63]:
# Using the optimum alpha for developing the model
ridge_reg = Ridge(alpha=rscore.argmax())
ridge_reg.fit(X_train,y_train)
yhat = ridge_reg.predict(X_train)
r2_score(y_train, yhat)

0.38945890777807923

In [64]:
yhat_test = ridge_reg.predict(X_test)
r2_score(y_test, yhat_test)

0.6193733022118144

In [65]:
# Determining the mean squared loss of the model
mean_squared_error(y_test, yhat_test)

755884824.5466051

In [66]:
# Determing the optimum alpha for the lasso regression model
from sklearn.linear_model import Lasso
rscore = []
for alpha in range(5000):
    lasso_reg = Lasso(alpha=alpha)
    lasso_reg.fit(X_train, y_train)
    yhat = lasso_reg.predict(X_train)
    rscore_ = r2_score(y_train, yhat)
    rscore.append(rscore_)
rscore = np.array(rscore)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.378e+12, tolerance: 1.318e+09 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, po

In [67]:
# Making a plot of the rscore of each alpha from 0 to 5000
plt.plot(rscore)
plt.xlabel("Alpha")
plt.ylabel("rscore")
plt.show()
plt.clf()

<Figure size 432x288 with 0 Axes>

In [68]:
lasso_reg = Lasso(alpha=rscore.argmax())
lasso_reg.fit(X_train, y_train)
yhat = lasso_reg.predict(X_train)
r2_score(y_train, yhat)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.378e+12, tolerance: 1.318e+09 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, po

0.389458907778079

In [69]:
yhat_test = lasso_reg.predict(X_test)
r2_score(y_test, yhat_test)

0.619373302211814

In [70]:
mean_squared_error(y_test, yhat_test)

755884824.5466061

In [71]:
Z = df2.drop(["lower_salary", "upper_salary"], axis = "columns")

In [72]:
Z.isna().sum()

title scraped for    0
Country              0
Position             0
year_of_exp          0
contract_type        0
eligibility          0
R                    0
Spark                0
C++                  0
Java                 0
SQL                  0
C                    0
Python               0
D                    0
AWS                  0
Git                  0
ML                   0
CSS                  0
HTML                 0
Excel                0
Go                   0
PostgreSQL           0
Vi                   0
client               0
ETL                  0
Machine Learning     0
                     0
SDLC                 0
dtype: int64

In [73]:
yhat_test = lasso_reg.predict(Z)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but Lasso was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [74]:
for i in range(X.shape[0]):
    Z.loc[i,["lower_salary", "upper_salary"]] = yhat_test[i]
    i +=1

In [75]:
Z.groupby(["Country", "title scraped for", "Position"])[["lower_salary","upper_salary"]].agg(np.mean)

lower_salary   upper_salary
Country title scraped for Position                              
0.0     0.0               0.0        62775.381998   84603.475595
                          1.0        55004.205749   76712.575743
                          2.0        46926.075381   68554.532395
        1.0               0.0        69757.737221   92649.226401
                          1.0        61075.170355   82822.431422
                          2.0        60219.219911   83852.916129
        2.0               0.0        75545.425227   99093.377121
                          1.0        68984.393525   92117.817007
                          2.0        65464.223963   88976.550169
        3.0               0.0        81738.769926  106697.540709
                          1.0        70724.796802   93399.704491
                          2.0        69300.807563   94351.314835
        4.0               0.0        91072.539970  117649.523489
                          1.0        85345.704487  111895.791122
                          2.0        79518.562922  105234.956550
2.0     0.0               1.0       115851.518747  146794.250727
                          2.0       115682.606272  147831.709835
        1.0               1.0       117269.491174  148447.431820
                          2.0       117089.652283  149472.689223
        2.0               0.0       137529.506981  172547.378386
                          1.0       133341.837255  169582.255058
                          2.0       129178.994646  165937.873283
        3.0               1.0       117690.828240  149057.362823
                          2.0       117510.989348  150082.620225
        4.0               0.0       150583.446898  187024.316202
                          1.0       143297.451717  179549.879032
                          2.0       134631.600807  169814.615520
3.0     0.0               0.0       159447.262888  200734.866853
                          1.0       151763.274973  193530.564046
                          2.0       150602.982396  193466.638062
        1.0               0.0       165690.710586  207668.391542
                          1.0       160472.314536  203039.836185
                          2.0       155035.783453  197224.688111
        2.0               0.0       171785.386371  214291.966149
                          1.0       169588.436626  213677.871275
                          2.0       166068.075907  210464.759936
        3.0               0.0       176095.085035  219112.103109
                          1.0       169727.938085  213128.233828
                          2.0       163370.444388  206206.354531
        4.0               0.0       184487.703232  228548.647479
                          1.0       179166.043801  223656.416258
                          2.0       172496.256117  217059.685345

In [76]:
XX

title scraped for  Country  Position  year_of_exp  contract_type  \
0                   3.0      0.0       0.0     1.000000           -1.0   
1                   3.0      0.0       0.0     1.000000            1.0   
2                   3.0      0.0       0.0     2.000000            1.0   
3                   3.0      0.0       1.0     4.247574           -1.0   
4                   3.0      0.0       0.0     2.022137            1.0   
...                 ...      ...       ...          ...            ...   
5639                3.0      2.0       2.0     9.000000           -1.0   
5640                3.0      2.0       2.0     9.000000           -1.0   
5641                3.0      2.0       2.0     9.000000           -1.0   
5642                3.0      2.0       2.0     9.000000           -1.0   
5643                3.0      2.0       2.0     9.000000           -1.0   

      eligibility    R  Spark  C++  Java  ...  HTML  Excel   Go  PostgreSQL  \
0             3.0  0.0    0.0  0.0   0.0  ...   0.0    0.0  0.0         0.0   
1             3.0  0.0    0.0  0.0   0.0  ...   0.0    0.0  0.0         0.0   
2             3.0  0.0    0.0  0.0   0.0  ...   0.0    0.0  0.0         0.0   
3             1.0  0.0    0.0  0.0   0.0  ...   0.0    0.0  0.0         0.0   
4             2.0  0.0    0.0  0.0   0.0  ...   0.0    0.0  0.0         0.0   
...           ...  ...    ...  ...   ...  ...   ...    ...  ...         ...   
5639          1.0  0.0    0.0  0.0   0.0  ...   0.0    0.0  0.0         0.0   
5640          1.0  0.0    0.0  0.0   0.0  ...   0.0    0.0  0.0         0.0   
5641          1.0  0.0    0.0  0.0   0.0  ...   0.0    0.0  0.0         0.0   
5642          1.0  0.0    0.0  0.0   0.0  ...   0.0    0.0  0.0         0.0   
5643          1.0  0.0    0.0  0.0   0.0  ...   0.0    0.0  0.0         0.0   

       Vi  client  ETL  Machine Learning       SDLC  
0     0.0     0.0  0.0               0.0  0.0   0.0  
1     0.0     0.0  0.0               0.0  0.0   0.0  
2     0.0     0.0  0.0               0.0  0.0   0.0  
3     0.0     0.0  0.0               0.0  0.0   0.0  
4     0.0     0.0  0.0               0.0  0.0   0.0  
...   ...     ...  ...               ...  ...   ...  
5639  0.0     0.0  0.0               0.0  0.0   0.0  
5640  0.0     0.0  0.0               0.0  0.0   0.0  
5641  0.0     0.0  0.0               0.0  0.0   0.0  
5642  0.0     0.0  0.0               0.0  0.0   0.0  
5643  0.0     0.0  0.0               0.0  0.0   0.0  

[5644 rows x 28 columns]

In [77]:
def get_weights_df(model, feat, col_name):
#this function returns the weight of every feature
    weights_lower_salary = pd.Series(model.coef_[0], feat.columns).sort_values()
    weights_lower_salary_df = pd.DataFrame(weights_lower_salary).reset_index()
    weights_lower_salary_df.columns = ['Features', col_name]
    weights_lower_salary_df[col_name].round(3)
    
    weights_upper_salary = pd.Series(model.coef_[1], feat.columns).sort_values()
    weights_upper_salary_df = pd.DataFrame(weights_upper_salary).reset_index()
    weights_upper_salary_df.columns = ['Features', col_name]
    weights_upper_salary_df[col_name].round(3)
    
    weights_df = weights_lower_salary_df.merge(weights_upper_salary_df,how = 'inner', on="Features")
    return weights_df
lasso_reg_weights = get_weights_df(lasso_reg, XX, 'lasso_reg_Weight_lower_salary')

In [78]:
lasso_reg_weights

Features  lasso_reg_Weight_lower_salary_x  \
0         year_of_exp                     -2812.108006   
1            Position                      -179.838892   
2                   D                         0.000000   
3    Machine Learning                         0.000000   
4                 ETL                         0.000000   
5              client                         0.000000   
6                  Vi                         0.000000   
7          PostgreSQL                         0.000000   
8                  Go                         0.000000   
9               Excel                         0.000000   
10               HTML                         0.000000   
11                CSS                         0.000000   
12                 ML                         0.000000   
13                Git                         0.000000   
14               SDLC                         0.000000   
15                                            0.000000   
16             Python                         0.000000   
17                  C                         0.000000   
18                SQL                         0.000000   
19               Java                         0.000000   
20                C++                         0.000000   
21              Spark                         0.000000   
22                  R                         0.000000   
23                AWS                         0.000000   
24      contract_type                      1946.077618   
25        eligibility                      4514.793102   
26  title scraped for                      6233.677036   
27            Country                     32529.412912   

    lasso_reg_Weight_lower_salary_y  
0                      -3140.326039  
1                       1025.257403  
2                          0.000000  
3                          0.000000  
4                          0.000000  
5                          0.000000  
6                          0.000000  
7                          0.000000  
8                          0.000000  
9                          0.000000  
10                         0.000000  
11                         0.000000  
12                         0.000000  
13                         0.000000  
14                         0.000000  
15                         0.000000  
16                         0.000000  
17                         0.000000  
18                         0.000000  
19                         0.000000  
20                         0.000000  
21                         0.000000  
22                         0.000000  
23                         0.000000  
24                      3698.173192  
25                      7040.797139  
26                      7030.955541  
27                     39112.341313